In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
from shapely.geometry import Polygon, MultiPolygon
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array
import matplotlib as mpl
from matplotlib.colors import TwoSlopeNorm, LogNorm
from cmcrameri import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle 
import matplotlib.ticker as mticker
import sklearn
import importlib
import cartopy.crs as ccrs

import sys
sys.path.append('../')
import snowFun

In [2]:
import importlib
importlib.reload(snowFun)

<module 'snowFun' from 'C:\\Users\\lzell\\OneDrive - Colostate\\Desktop\\AGVA\\scripts\\Figures\\..\\snowFun\\__init__.py'>

In [3]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_climate = os.path.join(folder_AGVA, "Climate")

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi, include_fields=['RGIId', 'geometry'])

# open the big data csv
path_data = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Final_Data_Table.csv')
data_df = pd.read_csv(path_data)

# open the glacier-specific future ela changes
# path_dela = os.path.join(folder_climate, 'CMIP', "rgi_delta_ELA.csv")
# dela_df = pd.read_csv(path_dela)

# # open glacier-specific AAR changes
# path_daar = os.path.join(folder_save, 'Average AAs', 'CMIP_derived_AARs.csv')

# subset rgi to the glaciers we have data for
rgi_gdf = rgi_gdf[ rgi_gdf['RGIId'].isin(data_df['RGIId']) ]
# print(rgi_gdf.columns)

# merge columns
all_data_gdf = gpd.GeoDataFrame(pd.merge(data_df, rgi_gdf, on="RGIId"))
# all_data_gdf = gpd.GeoDataFrame(pd.merge(all_data_gdf, dela_df, on="RGIId"))

# replace -9999 with nan
all_data_gdf[all_data_gdf==-9999] = np.nan

# what columns are available?
print(all_data_gdf.shape)
print(all_data_gdf.columns.values)

(3002, 82)
['RGIId' 'GLIMSId' 'BgnDate' 'EndDate' 'CenLon' 'CenLat' 'O1Region'
 'O2Region' 'Area' 'Zmin' 'Zmax' 'Zmed' 'Slope' 'Aspect' 'Lmax' 'Status'
 'Connect' 'Form' 'TermType' 'Surging' 'Linkages' 'Name' 'O3Region'
 'O3_name' 'ela_2018' 'ela_2019' 'ela_2020' 'ela_2021' 'ela_2022'
 'aar_2018' 'aar_2019' 'aar_2020' 'aar_2021' 'aar_2022' 'off_glacier_2018'
 'off_glacier_2019' 'off_glacier_2020' 'off_glacier_2021'
 'off_glacier_2022' 'date_2018' 'date_2019' 'date_2020' 'date_2021'
 'date_2022' 'ela_avg_product' 'aar_avg_product' 'aar_infill_avg_product'
 'off_glacier_avg_product' 'percent_observed_average_product' 'dhdt_1520'
 'dhdt_1020' 'distance_from_ocean' 'hyps_ela' 'hyps_aar' 'hyps_aar_20'
 'hyps_aar_40' 'hyps_aar_60' 'hyps_aar_80' 'hyps_aar_100' 'hyps_aar_120'
 'hyps_aar_140' 'hyps_aar_160' 'hyps_aar_180' 'hyps_aar_200'
 'hyps_aar_220' 'hyps_aar_240' 'hyps_aar_260' 'hyps_aar_280'
 'hyps_aar_300' 'hyps_aar_320' 'hyps_aar_340' 'hyps_aar_360'
 'hyps_aar_380' 'hyps_aar_400' 'hyps_a

First table:
Modern ELA, Modern AAR, Modern AAR (hyp), future ELA, future AAR (hyp)

In [4]:
# get the data for the first row (entire region average)
total_area = np.nansum(all_data_gdf['Area'])
modern_region_ela = np.nanmedian(all_data_gdf['ela_avg_product'])
modern_region_aa = np.nansum(all_data_gdf['aar_infill_avg_product']*all_data_gdf['Area'])
modern_region_aar = modern_region_aa/total_area
modern_region_aa_hyp = np.nansum(all_data_gdf['hyps_aar']*all_data_gdf['Area'])
modern_region_aar_hyp = modern_region_aa_hyp/total_area
future_region_ela = np.nanmedian(all_data_gdf['ela_avg_product']+all_data_gdf['delta_ela_cmip'])
future_region_aa = np.nansum(all_data_gdf['hyps_aar_cmip']*all_data_gdf['Area'])
future_region_aar = future_region_aa/total_area

alaska_row = {'Region':'All',
              'Modern ELA (m)': round(modern_region_ela),
              'Modern AAR': round(modern_region_aar,2),
              'Modern AAR (hypsometric)': round(modern_region_aar_hyp,2),
              'Future ELA (m)': round(future_region_ela),
              'Future AAR (hypsometric)': round(future_region_aar,2)}

alaska_row

{'Region': 'All',
 'Modern ELA (m)': 1750,
 'Modern AAR': 0.41,
 'Modern AAR (hypsometric)': 0.44,
 'Future ELA (m)': 1936,
 'Future AAR (hypsometric)': 0.33}

In [5]:
# now we have to do this for each subregion (o2 and o3)

In [6]:
# get hypsometric aa, aar for each 03region
all_data_gdf['modern_aa'] = all_data_gdf['aar_infill_avg_product']*all_data_gdf['Area']
all_data_gdf['hyps_aa'] = all_data_gdf['hyps_aar']*all_data_gdf['Area']
all_data_gdf['hyps_cmip_aa'] = all_data_gdf['hyps_aar_cmip']*all_data_gdf['Area']
all_data_gdf['future_ela_cmip'] = all_data_gdf['ela_avg_product']+all_data_gdf['delta_ela_cmip']

o3_df = pd.DataFrame(all_data_gdf.groupby('O3Region')['Area'].sum().rename('area'))

o3_df['Modern ELA (m)'] = all_data_gdf.groupby('O3Region')[f'ela_avg_product'].median()

o3_df['modern_aa'] = all_data_gdf.groupby('O3Region')[f'modern_aa'].sum()
o3_df['Modern AAR'] = o3_df[f'modern_aa']/o3_df['area']

o3_df['modern_aa_hyps'] = all_data_gdf.groupby('O3Region')[f'hyps_aa'].sum()
o3_df['Modern AAR (hypsometric)'] = o3_df[f'modern_aa_hyps']/o3_df['area']

o3_df['Future ELA (m)'] = all_data_gdf.groupby('O3Region')[f'future_ela_cmip'].median()

o3_df[f'future_aa_hyps'] = all_data_gdf.groupby('O3Region')[f'hyps_cmip_aa'].sum()
o3_df[f'Future AAR (hypsometric)'] = o3_df[f'future_aa_hyps']/o3_df['area']

o3_df['Region'] = [f'O3-{str(i).zfill(2)}' for i in range(17)]

# remove the glaciers in O3Region=0 (these are glaciers that don't fit into any of the regions)
o3_df = o3_df.drop(0, axis=0).reset_index()

# select only the columns we want
o3_df = o3_df[alaska_row.keys()]
o3_df

,Region,Modern ELA (m),Modern AAR,Modern AAR (hypsometric),Future ELA (m),Future AAR (hypsometric)
0,O3-01,1410.0,0.257470,0.258158,1560.5,0.139362
1,O3-02,1482.0,0.314612,0.327354,1658.0,0.144577
2,O3-03,1840.0,0.310326,0.329089,1998.0,0.229708
3,O3-04,1830.0,0.325720,0.338754,1966.0,0.265401
4,O3-05,2040.0,0.304095,0.341447,2166.0,0.253579
5,O3-06,2380.0,0.494388,0.506583,2522.0,0.409879
6,O3-07,2015.0,0.384278,0.409916,2137.0,0.205324
7,O3-08,1197.5,0.408871,0.476040,1329.5,0.297741
8,O3-09,1170.0,0.408720,0.472928,1301.0,0.302939
9,O3-10,1530.0,0.380456,0.406989,1667.0,0.305556


In [7]:
# do the same for o2 regions

o2_df = pd.DataFrame(all_data_gdf.groupby('O2Region')['Area'].sum().rename('area'))

o2_df['Modern ELA (m)'] = all_data_gdf.groupby('O2Region')[f'ela_avg_product'].median()

o2_df['modern_aa'] = all_data_gdf.groupby('O2Region')[f'modern_aa'].sum()
o2_df['Modern AAR'] = o2_df[f'modern_aa']/o2_df['area']

o2_df['modern_aa_hyps'] = all_data_gdf.groupby('O2Region')[f'hyps_aa'].sum()
o2_df['Modern AAR (hypsometric)'] = o2_df[f'modern_aa_hyps']/o2_df['area']

o2_df['Future ELA (m)'] = all_data_gdf.groupby('O2Region')[f'future_ela_cmip'].median()

o2_df[f'future_aa_hyps'] = all_data_gdf.groupby('O2Region')[f'hyps_cmip_aa'].sum()
o2_df[f'Future AAR (hypsometric)'] = o2_df[f'future_aa_hyps']/o2_df['area']

o2_df['Region'] = [f'O2-{str(i).zfill(2)}' for i in range(2,7)]

# remove the glaciers in O3Region=0 (these are glaciers that don't fit into any of the regions)
o2_df = o2_df.reset_index()

# # select only the columns we want
o2_df = o2_df[alaska_row.keys()]
o2_df

,Region,Modern ELA (m),Modern AAR,Modern AAR (hypsometric),Future ELA (m),Future AAR (hypsometric)
0,O2-02,1880.0,0.370275,0.387705,2025.0,0.283161
1,O2-03,1410.0,0.257470,0.258158,1560.5,0.139362
2,O2-04,1380.0,0.390402,0.430595,1521.0,0.301154
3,O2-05,2040.0,0.447476,0.476605,2160.0,0.417715
4,O2-06,1750.0,0.403861,0.435859,1967.5,0.237175


In [8]:
# concat them together
final_df = pd.concat([pd.DataFrame([alaska_row]),o2_df,o3_df]).reset_index().drop('index', axis=1)
final_df["Modern ELA (m)"] = np.round(final_df["Modern ELA (m)"]).astype(int)
final_df["Modern AAR"] = np.round(final_df["Modern AAR"],2)
final_df["Modern AAR (hypsometric)"] = np.round(final_df["Modern AAR (hypsometric)"],2)
final_df["Future ELA (m)"] = np.round(final_df["Future ELA (m)"]).astype(int)
final_df["Future AAR (hypsometric)"] = np.round(final_df["Future AAR (hypsometric)"],2)
final_df

,Region,Modern ELA (m),Modern AAR,Modern AAR (hypsometric),Future ELA (m),Future AAR (hypsometric)
0,All,1750,0.41,0.44,1936,0.33
1,O2-02,1880,0.37,0.39,2025,0.28
2,O2-03,1410,0.26,0.26,1560,0.14
3,O2-04,1380,0.39,0.43,1521,0.30
4,O2-05,2040,0.45,0.48,2160,0.42
5,O2-06,1750,0.40,0.44,1968,0.24
6,O3-01,1410,0.26,0.26,1560,0.14
7,O3-02,1482,0.31,0.33,1658,0.14
8,O3-03,1840,0.31,0.33,1998,0.23
9,O3-04,1830,0.33,0.34,1966,0.27


In [9]:
out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'supplement_table_1.csv')
final_df.to_csv(out_path, index=False)

lets do table S2 now

In [10]:
# get the data for the first row (entire region average ela/aar each year)
alaska_row = {'Region':'All'}

for y in [2018,2019,2020,2021,2022]:
    
    # subset to rows that have usable data this year
    subset = all_data_gdf[~np.isnan(all_data_gdf[f'ela_{y}'])]
    
    # calculate ela, aar
    ela_y = np.nanmedian(subset[f'ela_{y}'])
    aar_y = np.nansum(subset[f'aar_{y}']*subset['Area'])/np.nansum(subset['Area'])
    
    alaska_row[f'ELA {y}'] = round(ela_y)
    alaska_row[f'AAR {y}'] = round(aar_y,2)
    print(round(ela_y), round(aar_y,2))

alaska_row

1800 0.37
1940 0.25
1660 0.45
1568 0.49
1660 0.47


{'Region': 'All',
 'ELA 2018': 1800,
 'AAR 2018': 0.37,
 'ELA 2019': 1940,
 'AAR 2019': 0.25,
 'ELA 2020': 1660,
 'AAR 2020': 0.45,
 'ELA 2021': 1568,
 'AAR 2021': 0.49,
 'ELA 2022': 1660,
 'AAR 2022': 0.47}

In [11]:
# get these for each 03region
o3_rows = []
for r in range(1,17):
    
    # subset to o3 region
    subset_o3 = all_data_gdf[all_data_gdf['O3Region']==r]
    
    this_row = {'Region':f"O3-{str(r).zfill(2)}"}
    for y in [2018,2019,2020,2021,2022]:
    
        # subset to rows that have usable data this year
        subset = subset_o3[~np.isnan(subset_o3[f'ela_{y}'])]

        # calculate ela, aar
        ela_y = np.nanmedian(subset[f'ela_{y}'])
        aar_y = np.nansum(subset[f'aar_{y}']*subset['Area'])/np.nansum(subset['Area'])

        this_row[f'ELA {y}'] = round(ela_y)
        this_row[f'AAR {y}'] = round(aar_y,2)
    o3_rows.append(this_row)

In [12]:
# get these for each 02region
o2_rows = []
for r in range(2,7):
    
    # subset to o2 region
    subset_o2 = all_data_gdf[all_data_gdf['O2Region']==r]
    
    this_row = {'Region':f"O2-{str(r).zfill(2)}"}
    for y in [2018,2019,2020,2021,2022]:
    
        # subset to rows that have usable data this year
        subset = subset_o2[~np.isnan(subset_o2[f'ela_{y}'])]

        # calculate ela, aar
        ela_y = np.nanmedian(subset[f'ela_{y}'])
        aar_y = np.nansum(subset[f'aar_{y}']*subset['Area'])/np.nansum(subset['Area'])

        this_row[f'ELA {y}'] = round(ela_y)
        this_row[f'AAR {y}'] = round(aar_y,2)
    o2_rows.append(this_row)

In [13]:
# concat them together
final_df = pd.DataFrame( [alaska_row]+o2_rows+o3_rows )
final_df

out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'supplement_table_2.csv')
final_df.to_csv(out_path, index=False)